In [1]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from afinn import Afinn
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("Reviews.csv")
df = df[['Score', 'Text']].dropna()
df.head()  

,Score,Text
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,4,This is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...


In [3]:
def label_sentiment(score):
    if score >= 4:
        return "positive"
    elif score == 3:
        return "neutral"
    else:
        return "negative"

df['original_sentiment'] = df['Score'].apply(label_sentiment)
df['original_sentiment'].value_counts()

original_sentiment
positive    443777
negative     82037
neutral      42640
Name: count, dtype: int64

In [5]:
pip install tqdm


Note: you may need to restart the kernel to use updated packages.


In [7]:
from tqdm import tqdm
tqdm.pandas()

af = Afinn()

df['afinn_score'] = df['Text'].progress_apply(lambda x: af.score(x))

df['afinn_sentiment'] = df['afinn_score'].apply(
    lambda x: 'positive' if x > 0 else 'negative' if x < 0 else 'neutral'
)

df[['afinn_score', 'afinn_sentiment']].head()


100%|██████████| 568454/568454 [27:47<00:00, 340.84it/s] 


,afinn_score,afinn_sentiment
0,16.0,positive
1,-2.0,negative
2,3.0,positive
3,3.0,positive
4,9.0,positive


In [9]:
# Preprocessing setup
import nltk
nltk.download('punkt')
nltk.download('stopwords')

tqdm.pandas()

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess(text):
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    filtered = [stemmer.stem(w) for w in tokens if w not in stop_words and w.isalpha()]
    return ' '.join(filtered)

df['processed_text'] = df['Text'].progress_apply(preprocess)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 568454/568454 [16:54<00:00, 560.50it/s] 


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['processed_text'])

X.shape


(568454, 5000)

In [13]:
from sklearn.model_selection import train_test_split

y = df['original_sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set:", X_train.shape)
print("Test set    :", X_test.shape)


Training set: (454763, 5000)
Test set    : (113691, 5000)


In [17]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

nb = MultinomialNB()
nb.fit(X_train, y_train)

print("Naive Bayes Results:\n")
print(classification_report(y_test, nb.predict(X_test)))

Naive Bayes Results:

              precision    recall  f1-score   support

    negative       0.85      0.25      0.38     16181
     neutral       0.53      0.00      0.00      8485
    positive       0.81      1.00      0.90     89025

    accuracy                           0.81    113691
   macro avg       0.73      0.41      0.43    113691
weighted avg       0.80      0.81      0.76    113691



In [21]:
from sklearn.svm import LinearSVC

svm = LinearSVC()
svm.fit(X_train, y_train)

print("SVM Results:\n")
print(classification_report(y_test, svm.predict(X_test)))

SVM Results:

              precision    recall  f1-score   support

    negative       0.72      0.66      0.69     16181
     neutral       0.58      0.10      0.17      8485
    positive       0.89      0.97      0.93     89025

    accuracy                           0.86    113691
   macro avg       0.73      0.58      0.60    113691
weighted avg       0.84      0.86      0.84    113691



### Discussion: Sentiment Classification Models – AFINN, Naive Bayes, and SVM

In this study, we applied three different methods for sentiment analysis on the Amazon Fine Food Reviews dataset: a lexicon-based model (AFINN), Naive Bayes, and Support Vector Machine (SVM).

**AFINN Lexicon-Based Model:**
AFINN uses a predefined dictionary of words with sentiment scores ranging from -5 to +5. It is simple to implement and does not require training data. However, it lacks context sensitivity and struggles with sarcasm, negations, and words outside its dictionary. In large datasets, AFINN tends to overpredict "positive" sentiment due to the vocabulary bias in the lexicon.

**Naive Bayes Classifier:**
Naive Bayes performed reasonably well overall with an accuracy of 81%, but it showed poor recall for the "negative" and "neutral" classes. This is because Naive Bayes is sensitive to class imbalance and tends to favor the dominant class, which in this dataset is "positive".

**Support Vector Machine (SVM):**
SVM achieved the best performance with an accuracy of 86%, strong precision, and recall across all sentiment categories. It was especially effective for the "positive" class (F1-score of 0.93) and improved significantly over Naive Bayes in detecting "negative" and "neutral" sentiments. This confirms that SVM is a strong choice for high-dimensional text data like TF-IDF.

**Conclusion:**
While AFINN offers quick and interpretable results, SVM is the most robust and balanced model for sentiment classification in this dataset. Naive Bayes is acceptable but less reliable in imbalanced class scenarios.

*Name: Muhammad Hakimi bin Azizi*  
*Student ID: SW01082355*

*Name: Izzat Hatta bin Azizi*  
*Student ID: SW01082390*


In [26]:
df.to_csv("afinn_sentiment_output.csv", index=False)
